In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from pathlib import Path
from sklearn.model_selection import train_test_split

import tensorflow as tf
import os
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [3]:
train_df="/kaggle/input/surface-crack/train"
test_df="/kaggle/input/surface-crack/test"
valid_df="/kaggle/input/surface-crack/valid"

In [4]:
train_data = train_gen.flow_from_directory(
    train_df,
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,

 
)

val_data = train_gen.flow_from_directory(
    valid_df,
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True

   

)

test_data = train_gen.flow_from_directory(
    test_df,
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=False,

    
)

Found 600 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,RMSprop, Adagrad

from keras.layers import BatchNormalization

In [6]:
 def random_invert_img(x, p=0.6):
    if tf.random.uniform([]) < p:
        x = (255 - x)
    else:
        x
    return x
 def random_inv(factor = 0.65):
    return tf.keras.layers.Lambda(lambda x:random_invert_img(x, factor)) 

In [ ]:
model = Sequential()

model.add(tf.keras.layers.RandomContrast(0.2,0.7))
model.add(tf.keras.layers.RandomCrop(120,120))
model.add(tf.keras.layers.RandomRotation(0.2))
model.add(tf.keras.layers.RandomFlip("horizontal_and_vertical"))
model.add(tf.keras.layers.RandomContrast(0.2,0.7))
model.add(tf.keras.layers.RandomHeight(0.3))
model.add(random_inv())

model.add(Conv2D(32,3,padding="same", activation="relu", input_shape = (120,120,3)))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(128, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dropout(0.5))

model.add(BatchNormalization())
model.add(Dense(1, activation="sigmoid"))


model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [7]:
inputs = tf.keras.Input(shape=(120, 120, 3))
x=tf.keras.layers.RandomContrast(0.2,0.7)
x=tf.keras.layers.RandomCrop(120,120)
x=tf.keras.layers.RandomRotation(0.2)
x=tf.keras.layers.RandomFlip("horizontal_and_vertical")
x=tf.keras.layers.RandomHeight(0.3)
x=random_inv()





x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print(model.summary())

2023-01-13 13:45:12.480958: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-13 13:45:12.481918: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-13 13:45:12.483297: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-13 13:45:12.484095: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-13 13:45:12.484834: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 120, 120, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 118, 118, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 59, 59, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 57, 57, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 128)       0     

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [9]:
filepath="we111ights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True)
callbacks_list = [checkpoint, es]

In [10]:


history = model.fit(train_data,validation_data=val_data,epochs = 100,callbacks=callbacks_list ,verbose=1)

2023-01-13 13:45:14.850685: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2023-01-13 13:45:16.215124: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


19/19 [==============================] - 6s 131ms/step - loss: 0.6585 - accuracy: 0.6283 - val_loss: 0.6733 - val_accuracy: 0.5000

Epoch 00001: val_loss improved from inf to 0.67331, saving model to we111ights.best.hdf5
Epoch 2/100
19/19 [==============================] - 2s 112ms/step - loss: 0.5697 - accuracy: 0.6900 - val_loss: 0.7962 - val_accuracy: 0.5000

Epoch 00002: val_loss did not improve from 0.67331
Epoch 3/100
19/19 [==============================] - 2s 97ms/step - loss: 0.5135 - accuracy: 0.7533 - val_loss: 0.7031 - val_accuracy: 0.5000

Epoch 00003: val_loss did not improve from 0.67331
Epoch 4/100
19/19 [==============================] - 2s 101ms/step - loss: 0.4828 - accuracy: 0.7617 - val_loss: 0.6000 - val_accuracy: 0.5850

Epoch 00004: val_loss improved from 0.67331 to 0.60001, saving model to we111ights.best.hdf5
Epoch 5/100
19/19 [==============================] - 2s 114ms/step - loss: 0.4760 - accuracy: 0.7700 - val_loss: 0.6969 - val_accuracy: 0.5000

Epoch 000

In [11]:
fig = px.line( 
    history.history,
    y=['loss', 'val_loss'],
    labels={'index': "Epoch", 'value': "Loss"},
)

fig.show()

In [12]:
def evaluate_model(model, val_data):
    
    results = model.evaluate(val_data, verbose=0)
    loss = results[0]
    acc = results[1]
    print("    Test Loss: {:.5f}".format(loss))
    print("Test Accuracy: {:.2f}%".format(acc * 100))
    y_pred = np.squeeze((model.predict(test_data) >= 0.5).astype(np.int))
   
    clr = classification_report(test_data.labels, y_pred, target_names=["NEGATIVE", "POSITIVE"])
    print("Classification Report:\n----------------------\n", clr)
                                                                        



**classifcation report for Test data**

In [15]:
evaluate_model(model, test_data)

    Test Loss: 0.28394
Test Accuracy: 86.50%
Classification Report:
----------------------
               precision    recall  f1-score   support

    NEGATIVE       0.79      1.00      0.88       100
    POSITIVE       1.00      0.73      0.84       100

    accuracy                           0.86       200
   macro avg       0.89      0.86      0.86       200
weighted avg       0.89      0.86      0.86       200



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



**classifcation report for validation data**

In [16]:
evaluate_model(model, val_data)

    Test Loss: 0.14484
Test Accuracy: 95.00%
Classification Report:
----------------------
               precision    recall  f1-score   support

    NEGATIVE       0.79      1.00      0.88       100
    POSITIVE       1.00      0.73      0.84       100

    accuracy                           0.86       200
   macro avg       0.89      0.86      0.86       200
weighted avg       0.89      0.86      0.86       200



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [ ]:
y_pred.size